In [1]:
#doc

In [2]:
import gensim
import numpy as np

In [3]:
#Import job data
import pandas as pd
from pandas import DataFrame
import json

data = []

with open("/Users/ryancramer/Desktop/handshake_feed1.txt") as f:
    for line in f:
        data.append(json.loads(line))
        
df1 = pd.DataFrame(data)

data = []

with open("/Users/ryancramer/Desktop/courses.txt") as f:
    for line in f:
        data.append(json.loads(line))
        
df2 = pd.DataFrame(data)


df_skill = pd.read_csv("/Users/ryancramer/Desktop/Jupyter Notebook Files/SkillsSet5.csv")

skill5 = list()
df_skill = df_skill["0"]
for word in df_skill:
    #print(word)
    skill5.append(word)

#train_texts = df4['description']
jobs_text = df1['description']
courses_text = df2['description']

In [4]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        #sents = raw.split(".")
        #tokens = tokenizer.tokenize(raw)
        #text1 = []
        #for sent in sents:
        tokens = tokenizer.tokenize(raw)
            # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
            # stem tokens
        alpha_tokens = [i for i in stopped_tokens if i.isalpha() and len(i)<14]
            # alpha tokens only
            #stemmed_tokens = [p_stemmer.stem(i) for i in alpha_tokens]
            # add tokens to list
        texts.append((" ").join(alpha_tokens))
        #texts.append(text1)
    return texts

In [5]:
import spacy
nlp = spacy.load("/Users/ryancramer/opt/anaconda3/lib/python3.7/site-packages/en_core_web_lg/en_core_web_lg-2.3.1", disable=['parser', 'ner'])

nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [6]:
%%time

docJ = nlp.pipe(jobs_text)
docC = nlp.pipe(courses_text)

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 10 µs


In [7]:
#newwwwwwwww
all_sents_processed = []
for doc in docJ:
    sentences = (list(doc.sents))
    for sent in sentences:
        all_sents_processed.append(sent)
        
for doc in docC:
    sentences = (list(doc.sents))
    for sent in sentences:
        all_sents_processed.append(sent)

In [8]:
%%time
jobs_processed = preprocess_data(jobs_text)
courses_processed = preprocess_data(courses_text)


CPU times: user 1.77 s, sys: 102 ms, total: 1.87 s
Wall time: 2.08 s


In [9]:
%%time
corpus = []
for row in all_sents_processed:
        corpus_hold = list()
        for word in row:
            #word = word.lower()
            corpus_hold.append(str(word.lemma_.lower()))
        corpus.append(corpus_hold)
print(len(corpus))

171159
CPU times: user 4.73 s, sys: 691 ms, total: 5.42 s
Wall time: 5.85 s


In [10]:
from gensim.test.utils import get_tmpfile
from gensim.models import Doc2Vec




In [11]:
from gensim.models.doc2vec import TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus)]

In [12]:
%%time


modelD = Doc2Vec(documents, window=5, min_count=0, workers=4)

CPU times: user 2min 51s, sys: 20 s, total: 3min 11s
Wall time: 2min 52s


In [13]:
#model.save("D2VCorpus.model")

In [14]:
def ngram(doc_set):
    texts = []
    for i in doc_set:
        text_with_grams = []
        count = 1
        holder = ""
        words = i.split()
        for word in words:
            if count == 1:
                holder = word
            else:  
                text_with_grams.append(holder + " " + word)
                text_with_grams.append(holder)
                if count == len(words):
                    text_with_grams.append(word)
                holder = word
            count = count + 1
        texts.append(text_with_grams)
    return texts

In [15]:
#(BETTER)
def SkillConverter(jobs,skills):
    texts = []
    for job in jobs:
        skill_text = []
        skip_gram = False
        for word in job:
            #if skip_gram == True:
                #skip_gram = False
                #continue
            #elif skip_gram == False and word in skills:
                
                #skill_text.append(word)
                #skip_gram = True
            if word in skills:
                skill_text.append(word)
        texts.append((" ").join(skill_text))
    return texts

In [18]:
%%time
jobs_ngram = ngram(jobs_processed)
jobs_skills = SkillConverter(jobs_ngram,skill5)
courses_ngram = ngram(courses_processed)
courses_skills = SkillConverter(courses_ngram,skill5)

CPU times: user 9min 41s, sys: 8.51 s, total: 9min 49s
Wall time: 10min 37s


In [22]:
doc3 = nlp.pipe(jobs_skills)
#doc4 = nlp.pipe(courses_skills)

In [23]:
for job in doc3:
    print(modelD.wv[job])
    break

KeyError: "word 'employer' not in vocabulary"

In [20]:
#nOOOO
job_sumsD = []
for jobss in doc3:
    count = 0
    hold1 = np.empty(100)
    for word in jobss:
        #print(word.lemma_)
        if word.lemma_ in modelD.wv.vocab:
            hold1 = hold1 + modelD.wv[word.lemma_]
        else:
            print(word.lemma_)
        count = count +1
    if count != 0:
        hold1 = hold1 / count
    job_sumsD.append(hold1)

advertise
advertise
advertise
strut
spre
-PRON-
-PRON-
-PRON-
zenwork
administ
ongoe
ongoe
advertise
launder
acuman
-PRON-
launder
launder
launder
launder
launder
launder
-PRON-
ongoe
strut
strut
-PRON-
-PRON-
advertise
advertise
advertise


In [ ]:
#nOOO
course_sumsD = []
for jobss in doc4:
    count = 0
    hold1 = np.empty(100)
    for word in jobss:
        #print(word.lemma_)
        if word.lemma_ in modelD.wv.vocab:
            hold1 = hold1 + modelD.wv[word.lemma_]
        else:
            print(word.lemma_)
        count = count +1
    if count != 0:
        hold1 = hold1 / count
    course_sumsD.append(hold1)

In [ ]:
job_sums = []
for docum in jobs_skills:
    job_sums.append(model.wv[docum])
        
course_sums = []
for docum in courses_skills:
    course_sums.append(model.wv[docum])

In [ ]:
%%time
job_sums = []
for jobss in jobs_skills:
    count = 0
    hold1 = np.empty(100)
    for word in jobss.split():
        #print(word)
        hold1 = hold1 + model.wv[word]
        count = count +1
    if count != 0:
        hold1 = hold1 / count
    job_sums.append(hold1)

course_sums = []
for jobss in courses_skills:
    count = 0
    hold1 = np.empty(100)
    for word in jobss.split():
        #print(word)
        hold1 = hold1 + model.wv[word]
        count = count +1
    if count != 0:
        hold1 = hold1 / count
    course_sums.append(hold1)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
sim1 = 1-pairwise_distances(job_sums,course_sums, metric='cosine')
sim2 = 1-pairwise_distances(job_sums,course_sums, metric='euclidean')

In [ ]:
df_sim1 = pd.DataFrame(sim1)
df_sim1

In [ ]:
df_sim2 = pd.DataFrame(sim2)
df_sim2